## Simulation of  modified Stuart-Landau (MSL) model with (generalized) phase model

### Setup

In [1]:
using GeneralizedPhaseModel
using DifferentialEquations, PyPlot
using PyCall
axes_grid1 = pyimport("mpl_toolkits.axes_grid1")

PyObject <module 'mpl_toolkits.axes_grid1' from 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\mpl_toolkits\\axes_grid1\\__init__.py'>

### Model definition

In [2]:
dxdt(X, I, λ=5.0) = λ*(X[1]-I) - exp(I)*X[2]-λ*((X[1]-I)^2+X[2]^2)*(X[1]-I)
dydt(X, I, λ=5.0) = exp(I)*(X[1]-I) + λ*X[2]-λ*((X[1]-I)^2+X[2]^2)*X[2]
F(X, I) = [dxdt(X, I), dydt(X, I)]

D, N = 2, 2 # number of dimensions, units
Nθ = 1000
G(X, K) = K * ((1 .- eye(N)) * X[:, 1])
dt = 1e-3; T = 20; Nt = round(Int, T/dt)
trange = range(0, T, length=Nt)
alg = Tsit5();

In [3]:
@time Ts, ω, Xs = find_stable_periodic_solution(F, 0, D, Nθ, [1, 0], dt, alg, 2, 0.0, print_progress=false)
println("Ts=", Ts, " (sec) , ω=", ω, " (Hz)")

  7.425818 seconds (29.75 M allocations: 2.172 GiB, 5.04% gc time, 61.46% compilation time)
Ts=6.284 (sec) , ω=0.9998703544206853 (Hz)


In [6]:
# F, Imin, Imax, dI, D, Nθ, nothing, dt, alg, origin_val_idx, origin_thr
ωI, ζθI, ξθI, XsI = generalized_phase_sensitivity_func(F, -1.2, 1.2, 0.1, D, Nθ, nothing, dt, alg, 2, 0.0);

[1/3] Computing Xs(θ, I) and Z(θ, I)...100%|████████████| Time: 0:00:09m


[2/3] Computing ζ(θ, I)...
[3/3] Computing ξ(θ, I)...


In [5]:
function coupled_func!(dX, X, p, t)
    g, κ = p
    Iext = g(X) # input
    for i in 1:size(dX)[1]
        dX[i, :] = κ[i] * F(X[i, :], Iext[i])
    end
end

coupled_func! (generic function with 1 method)

### Run simulation

In [17]:
function all_simulation(N, D, ωI, ζθI, ξθI, XsI, coupled_func!, G, K, κ, initθ, alg=Tsit5(), NΘ=50, input_mode="I")
    @assert (input_mode == "I" || input_mode == "PQ") "input_mode must be `I` or `PQ`."
    
    g(X) = G(X, K)
    Xos, Θgos, Θcos = coupled_original_system(N, D, Nt, dt, XsI, g, coupled_func!, initθ, κ, alg)
    print("OK")
    Xcpm, Θcpm = coupled_conventinal_phase_model(N, D, Nt, dt, XsI, g, ωI, ζθI, Θcos[1, :], κ, alg)
    if input_mode == "I"
        IΘ = compute_IsΘ(g, N, D, XsI, NΘ)
        Xgpm, Θgpm = coupled_generalized_phase_model_I(N, D, Nt, dt, XsI, IΘ, ωI, ξθI, Θgos[1, :], κ, alg)
        return Xos, Xcpm, Xgpm, Θgos, Θcos, Θcpm, Θgpm, IΘ
    elseif input_mode == "PQ"
        QΘ = compute_QΘ(g, N, D, XsI, κ, ωI, NΘ)
        Xgpm, Θgpm = coupled_generalized_phase_model_PQ(N, D, Nt, dt, XsI, QΘ, ωI, ζθI, ξθI, Θgos[1, :], κ, g, alg)
        return Xos, Xcpm, Xgpm, Θgos, Θcos, Θcpm, Θgpm, QΘ
    end
end

all_simulation (generic function with 4 methods)

In [18]:
K = 0.5
κ = [1.0, 1.5]
initθ = [0, π]
NΘ = 50
input_mode = "I" # "I" or "PQ"

@time Xos, Xcpm, Xgpm, Θgos, Θcos, Θcpm, Θgpm, IΘ = all_simulation(
    N, D, ωI, ζθI, ξθI, XsI, coupled_func!, G, K, κ, initθ, alg, NΘ, input_mode);

OK

LoadError: UndefVarError: D not defined

### Results

In [9]:
plt.rc("text", usetex=true)
PyCall.PyDict(plt."rcParams")["font.size"] = 14
PyCall.PyDict(plt."rcParams")["xtick.direction"] = "in"
PyCall.PyDict(plt."rcParams")["ytick.direction"] = "in"
PyCall.PyDict(plt."rcParams")["xtick.minor.visible"] = true
PyCall.PyDict(plt."rcParams")["ytick.minor.visible"] = true
PyCall.PyDict(plt."rcParams")["xtick.top"] = true 
PyCall.PyDict(plt."rcParams")["ytick.right"] = true 
PyCall.PyDict(plt."rcParams")["font.family"] = "Arial"
PyCall.PyDict(plt."rcParams")["text.latex.preamble"] = [raw"\usepackage{amsmath}"];

sys:1: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.


In [10]:
ωt1 = (κ[1]*ω) .* trange
ωt2 = (κ[2]*ω) .* trange;
Δωt = ωt2 - ωt1;
Θrange = range(0, 2π, length=100)
Irange = 1.2:-0.1:-1.2;

In [11]:
figure(figsize=(16, 6), dpi=300)
gca().set_rasterization_zorder(1)
#### left ####
ax7 = subplot2grid((7, 8), (0, 0), rowspan=2, colspan=2)
ζ=ζθI(Θrange, Irange)'
contour(Θrange, Irange, ζ, 10, colors="white")
imshow(ζ, extent=(0, 2pi, -1.2, 1.2))
ylabel(L"$I$"); xlabel(L"$\theta$");
xticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
divider7 = axes_grid1.make_axes_locatable(ax7)
cax7 = divider7.append_axes("right", size="5%", pad=0.1)
cbar7 = colorbar(cax=cax7)
cbar7.set_label(L"$\zeta (\theta, I)$")
ax7.text(-0.15, 1.3, "(a)", fontsize=16, transform=ax7.transAxes, fontweight="bold", va="top")

ax8 = subplot2grid((7, 8), (2, 0), rowspan=2, colspan=2)
ξ=ξθI(Θrange, Irange)'
contour(Θrange, Irange, ξ, 10, colors="white")
imshow(ξ, extent=(0, 2pi, -1.2, 1.2))
ylabel(L"$I$"); xlabel(L"$\theta$");
xticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
divider8 = axes_grid1.make_axes_locatable(ax8)
cax8 = divider8.append_axes("right", size="5%", pad=0.1)
cbar8 = colorbar(cax=cax8)
cbar8.set_label(L"$\xi (\theta, I)$")
ax8.text(-0.15, 1.3, "(b)", fontsize=16, transform=ax8.transAxes, fontweight="bold", va="top")

ax9 = subplot2grid((7, 8), (4, 0), rowspan=3, colspan=2)
IΘmap=IΘ[1](Θrange, Θrange)
contour(Θrange, Θrange, IΘmap, 10, colors="white")
imshow(reverse(IΘmap, dims=1), extent=(0, 2pi, 0, 2pi))
xlabel(L"$\theta_1$"); ylabel(L"$\theta_2$");
xticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
yticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
divider9 = axes_grid1.make_axes_locatable(ax9)
cax9 = divider9.append_axes("right", size="5%", pad=0.1)
cbar9 = colorbar(cax=cax9)
cbar9.set_label(L"$Q_1\left(\theta_1, \theta_2\right)$")
ax9.text(-0.15, 1.2, "(c)", fontsize=16, transform=ax9.transAxes, fontweight="bold", va="top")

#### middle ####
ax1 = subplot2grid((7, 8), (0, 2), rowspan=3, colspan=3)
# title("Conventinal phase model")
plot(trange, Xos[:, 1, 1], label="original system", color="tab:orange")
plot(trange, Xcpm[:, 1, 1], "--", label="phase eq. [conventional]", color="tab:red")
plot(trange, Xos[:, 2, 1], label=" ", color="tab:green")
plot(trange, Xcpm[:, 2, 1], "--", label=" ", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+1.8); ylabel(L"$x_1(t), x_2(t)$"); 
plt.gca().axes.xaxis.set_ticklabels([])
ax1.text(-0.15, 1.15, "(d)", fontsize=16, transform=ax1.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=2, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

ax3=subplot2grid((7, 8), (3, 2), rowspan=2, colspan=3)
plot(trange, Θcos[:, 1]-ωt1, label="original system", color="tab:orange")
plot(trange, Θcpm[:, 1]-ωt1, "--", label="phase eq. [conventional]", color="tab:red")
plot(trange, Θcos[:, 2]-ωt2, label=" ", color="tab:green") 
plot(trange, Θcpm[:, 2]-ωt2, "--", label=" ", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+8); ylabel(L"$\theta(t)-\omega t$"); 
plt.gca().axes.xaxis.set_ticklabels([])
ax3.text(-0.15, 1.3, "(f)", fontsize=16, transform=ax3.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=2, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

ax5=subplot2grid((7, 8), (5, 2), rowspan=2, colspan=3)
plot(trange, Θcos[:, 2]-Θcos[:, 1]-Δωt, label="original system", color="tab:green")
plot(trange, Θcpm[:, 2]-Θcpm[:, 1]-Δωt, "--", label="phase eq. [conventional]", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+4); ylabel(L"$\Delta \theta(t)-\Delta \omega t$"); xlabel(L"$t$");
yticks([-4, -2,0,2])
ax5.text(-0.15, 1.3, "(h)", fontsize=16, transform=ax5.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=1, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

#### right ####
ax2=subplot2grid((7, 8), (0, 5), rowspan=3, colspan=3)
plot(trange, Xos[:, 1, 1], label="original system", color="tab:orange")
plot(trange, Xgpm[:, 1, 1], "--", label="phase eq. [proposed]", color="tab:red")
plot(trange, Xos[:, 2, 1], label=" ", color="tab:green")
plot(trange, Xgpm[:, 2, 1], "--", label=" ", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+1.8);  
#ylabel(L"$x_1(t), x_2(t)$"); 
plt.gca().axes.xaxis.set_ticklabels([])
ax2.text(-0.15, 1.15, "(e)", fontsize=16, transform=ax2.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=2, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

ax4=subplot2grid((7, 8), (3, 5), rowspan=2, colspan=3)
plot(trange, Θgos[:, 1]-ωt1, label="original system", color="tab:orange")
plot(trange, Θgpm[:, 1]-ωt1, "--", label="phase eq. [proposed]", color="tab:red")
plot(trange, Θgos[:, 2]-ωt2, label=" ", color="tab:green") 
plot(trange, Θgpm[:, 2]-ωt2, "--", label=" ", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+8); 
#ylabel(L"$\theta(t)-\omega t$"); 
plt.gca().axes.xaxis.set_ticklabels([])
ax4.text(-0.15, 1.3, "(g)", fontsize=16, transform=ax4.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=2, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

ax6=subplot2grid((7, 8), (5, 5), rowspan=2, colspan=3)
plot(trange, Θgos[:, 2]-Θgos[:, 1]-Δωt, label="original system", color="tab:green")
plot(trange, Θgpm[:, 2]-Θgpm[:, 1]-Δωt, "--", label="phase eq. [proposed]", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+4); 
yticks([-4, -2,0,2])
#ylabel(L"$\Delta \theta(t)-\Delta \omega t$"); 
xlabel(L"$t$");
ax6.text(-0.15, 1.3, "(i)", fontsize=16, transform=ax6.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=1, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

subplots_adjust(hspace=1, wspace=5)
#show()
savefig("fig1.svg")

LoadError: UndefVarError: IΘ not defined